# Analys av könsfördelningen bland styrelseordföranden i svenska aktiebolag
av [Anna Norberg](https://twitter.com/annanorberg)

Hur ser könsfördelningen ut bland styrelseordföranden för aktiebolag i landets kommuner? Nyhetsbyrån Siren har gjort en analys av Bolagsverkets öppna data om företrädare i svenska företag, som finns dokumenterad nedan.

I beräkningen ingår alla aktiebolag i Sverige i maj 2018 där en styrelseordförande finns utsedd.

Resultatet från analysen har rapporterats i en artikel som skickades ut till Nyhetsbyrån Sirens kunder den xx maj 2018.

## Så här gjorde vi
### Importera Pythons analysverktyg

In [13]:
import pandas as pd

### Läs in data från Bolagsverkets hemsida

In [14]:
ordf = pd.read_csv("http://www.bolagsverket.se/statistik/foretradare_historik.csv", engine='python', delimiter='\;')

In [15]:
ordf.head()

,Ar,Foretagsform,Lan,Kommun,PrivatPublikt,Arbetstagarrepresentant,Utlandsbosatt,Bosatt_EES,Kon,Alder_intervall,...,Antal_REVS,Antal_REVSL,Antal_REVST,Antal_REVT,Antal_SU,Antal_SVD,Antal_VD,Antal_VLE,Antal_VOF,Antal_VVD
0,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Kvinna,50 �r och �ldre,...,10,0,0,0,221,0,6,0,0,2
1,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Kvinna,Yngre �n 50 �r,...,18,1,0,0,180,0,8,0,0,0
2,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Man,50 �r och �ldre,...,51,11,0,0,197,0,100,0,0,3
3,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Man,Yngre �n 50 �r,...,214,0,0,0,128,0,87,0,0,5
4,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Org,Ok�nd,...,0,0,0,0,0,0,0,0,0,0


### Förbered datan för analys
Filtrera ut endast aktiebolag, aktuellt år och företag med fysiska företrädare. Summera antalet styrelseordföranden per kön och kommun.

In [16]:
aktiebolag = ordf[ordf.Foretagsform == "AB"]
ab_arton = aktiebolag[aktiebolag.Ar == 2018]
ab_arton_fys = ab_arton[ab_arton.JurFys == "F"]
ordf_ab = ab_arton_fys.groupby(["Kommun", "Kon"]).Antal_OF.sum().reset_index()
kommuner = ordf_ab.pivot(index="Kommun", columns="Kon")
pd.set_option('precision', 0)
kommuner.drop(kommuner.columns[[2]], axis=1, inplace=True)

Nu ser datasetet ut så här:

In [17]:
kommuner.head()

Antal_OF     
Kon               Kvinna  Man
Kommun                       
Ale kommun            39  172
Alings�s kommun       65  517
Alvesta kommun        23  171
Aneby kommun           3   59
Arboga kommun         23  114

### Beräkna totalt antal styrelseordföranden, samt andelen kvinnor och män

In [18]:
kommuner.loc['Summa']= kommuner.sum()
kommuner['Totalt'] = kommuner.sum(axis=1)
kommuner.columns = ['Kvinna', 'Man', 'Totalt']
kommuner["Andel kvinnor"] = (kommuner.Kvinna / kommuner.Totalt)*100
kommuner["Andel män"] = (kommuner.Man / kommuner.Totalt)*100
kommuner.head()

,Kvinna,Man,Totalt,Andel kvinnor,Andel män
Kommun,,,,,
Ale kommun,39,172,211,18,82
Alings�s kommun,65,517,582,11,89
Alvesta kommun,23,171,194,12,88
Aneby kommun,3,59,62,5,95
Arboga kommun,23,114,137,17,83


### Rangordna kommunerna
Här är andelen kvinnor lägst:

In [19]:
kommuner.sort_values("Andel kvinnor").head()

,Kvinna,Man,Totalt,Andel kvinnor,Andel män
Kommun,,,,,
�sele kommun,1,52,53,2,98
Grums kommun,2,45,47,4,96
Aneby kommun,3,59,62,5,95
Brom�lla kommun,5,88,93,5,95
Storfors kommun,2,32,34,6,94


Här är andelen kvinnor högst:

In [20]:
kommuner.sort_values("Andel kvinnor", ascending=False).head()

,Kvinna,Man,Totalt,Andel kvinnor,Andel män
Kommun,,,,,
Surahammar kommun,15,32,47,32,68
�verkalix kommun,11,27,38,29,71
Falun kommun,253,653,906,28,72
Dorotea kommun,10,26,36,28,72
Salem kommun,25,66,91,27,73


### Exportera det nya datasetet till csv

In [21]:
kommuner.to_csv("ordf.csv", decimal=',', sep=',', float_format='%.0f')